# 2.0 Parameter settings

In [ ]:
## 2.0.1 Library import
import configparser
from azureml.core import Workspace
from azureml.core.authentication import AzureCliAuthentication
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.pipeline.core._restclients.aeva.models.error_response import ErrorResponseException
from azureml.pipeline.core import Pipeline, PipelineEndpoint, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep

In [ ]:
## 2.0.2 Retrieve configuration file
config_ini = configparser.ConfigParser()
config_ini.read('./common/config.ini', encoding='utf-8')

## 2.0.3 Basic Azure parameters
subscription_id = config_ini.get('Azure', 'subscription_id')
resource_group = config_ini.get('Azure', 'resource_group')
workspace_name = config_ini.get('Azure', 'workspace_name')

## 2.0.4 Azure ML parameters
cluster_name = config_ini.get('AML', 'cluster_name')
vm_size = config_ini.get('AML', 'vm_size')
vm_location = config_ini.get('AML', 'vm_location')
managed_id = config_ini.get('AML', 'managed_id')

## 2.0.5 Parameters for image classification
TASK_TYPE=config_ini.get('AML', 'TASK_TYPE')
experiment_name_for_inference = config_ini.get('AML', 'experiment_name_for_inference')
pipelineName_for_inference = config_ini.get('AML', 'pipelineName_for_inference')
model_name = config_ini.get('AML', 'model_name')

# 2.1 Authentication

In [ ]:
## 2.1.1 Retrieve AML workspace with CLI authentication
cli_auth = AzureCliAuthentication()
ws = Workspace(subscription_id=subscription_id,
               resource_group=resource_group,
               workspace_name=workspace_name,
               auth=cli_auth)

# 2.2 Define computer target

`compute_target` will be used in inferring with generated deep learning model.

In [ ]:
## 2.2.1 Define computer target
try:
    compute_target = ws.compute_targets[cluster_name]
    print("Found existing compute target.")
except KeyError:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size=vm_size,
        idle_seconds_before_scaledown=600,
        min_nodes=0,
        max_nodes=4,
        location=vm_location,
        identity_type=managed_id,
    )
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(
    show_output=True, min_node_count=None, timeout_in_minutes=20
)

# 2.3 Run configuration

## 2.3.1 Run configuration

It connects `compute_target` with the other settings in executing AML pipeline.

In [ ]:
aml_run_config = RunConfiguration()
aml_run_config.target = compute_target

## 2.3.2 Add some packages for python environment

Please check #1.3.2 in [Training Notebook](./10.%20AML_pipeline_train.ipynb) for more detail.

In [ ]:
aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    python_version='3.7'
    ,conda_packages=['pandas'
                ,'scikit-learn'
                ,'numpy==1.20.1'
                ,'pycocotools==2.0.2'
                ]
    ,pip_packages=['azureml-sdk'
                ,'azureml-automl-core'
                ,'azureml-automl-dnn-vision==1.43.0'
                ]
    ,pin_sdk_version=False)

# 2.4 Generate AML pipeline for inferring

In [ ]:
## 2.4.1 Define python script and its arguments to be used
pipeline_param01 = PipelineParameter(name="pipeline_arg01", default_value="3")
pipeline_param02 = PipelineParameter(name="pipeline_arg02", default_value="3")
pipeline_param03 = PipelineParameter(name="pipeline_arg03", default_value="3")
pipeline_param04 = PipelineParameter(name="pipeline_arg04", default_value="3")
pipeline_param05 = PipelineParameter(name="pipeline_arg05", default_value="3")
pipeline_param06 = PipelineParameter(name="pipeline_arg06", default_value="3")

InferenceStep = PythonScriptStep(
    script_name="inference.py",
    arguments=[
        "--subscription_id",               pipeline_param01,
        "--resource_group",                pipeline_param02,
        "--workspace_name",                pipeline_param03,
        "--cluster_name",                  pipeline_param04,
        "--model_name",                    pipeline_param05,
        "--TASK_TYPE",                     pipeline_param06
    ],
    compute_target=compute_target,
    source_directory='.',
    runconfig=aml_run_config,
    allow_reuse = True,
)

## 2.4.2 Define Azure ML Pipeline

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[InferenceStep])

## 2.4.2.2 Work-around in some error

Please use the following cell, if you encounter some error. Please see #1.4.2.2 in [training notebook](./10.%20AML_pipeline_train.ipynb)

In [ ]:
import azureml
azureml._restclient.snapshots_client.SNAPSHOT_MAX_SIZE_BYTES = 1000000000

# 2.4.3 Publish the pipeline

As in #1.4.3 in [training notebook](./10.%20AML_pipeline_train.ipynb), the following pipeline will provide fixed URI, which enables you to use the same API target to trigger the pipeline.

In [ ]:
try:
    pipelineEndpoint = PipelineEndpoint.get(workspace=ws, name=pipelineName_for_inference)
except ErrorResponseException as ex:
    if "not found in workspace" in ex.message:
        pipelineEndpoint = None
    else:
        raise

if pipelineEndpoint is None:
    print('Pipeline does not exists, creating new: ' + pipelineName_for_inference)
    pipelineEndpoint = PipelineEndpoint.publish(workspace = ws
                                        ,name = pipelineName_for_inference
                                        ,pipeline=pipeline
                                        ,description="Inferrence Pipeline.")
else:
    print('Found existing pipeline ' + pipelineName_for_inference + ', adding new version.')
    published_pipeline = pipeline.publish(name = pipelineName_for_inference + "_Pipeline")
    pipelineEndpoint.add_default(published_pipeline)

# 2.4.4 Submit the pipeline

You can execute the pipeline by giving the parameters.

In [ ]:
pipeline.submit(experiment_name=experiment_name_for_inference,
                pipeline_parameters={"pipeline_arg01": subscription_id
                                    ,"pipeline_arg02": resource_group
                                    ,"pipeline_arg03": workspace_name
                                    ,"pipeline_arg04": cluster_name
                                    ,"pipeline_arg05": model_name
                                    ,"pipeline_arg06": TASK_TYPE                         
                                    })

In [ ]:
model_name